# Dados UFRN - Análise


## Descrição do projeto

## Descrição dos dados


## Autores


## Vídeo explicativo



Importando bibliotecas a serem usadas

In [2]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
#conda install -c conda-forge seaborn

Abertura de dados

In [3]:
#Carregando arquivo referente as unidades
data_unidades = pd.read_csv("Dados/unidades/unidades.csv", sep=';',
                            usecols=["id_unidade", "nome_unidade", "sigla", "municipio","id_unidade_gestora",
                                     "id_unidade_responsavel", "unidade_responsavel", "classificacao_unidade"])


#Selecionando somente os institutos e os centros
centro_ou_inst = [nome.startswith("CENTRO") or nome.startswith("INSTITUTO") for nome in data_unidades.nome_unidade]
data_unidades = data_unidades[centro_ou_inst]

#Carregando arquivo referente aos gastos
data_gastos = pd.read_csv("Dados/unidades/gastos-por-unidade.csv", 
                          sep=';',
                          usecols=["id_unidade", "natureza_despesa", "valor"])

#Convertendo coluna de String para float
data_gastos.valor = data_gastos.valor.replace('[R | ' ' | \$,.]', '', regex=True).astype(float) / 100

#Unindo as tabelas
dataM_unit_gas = pd.merge(data_unidades, data_gastos, on="id_unidade")

#Criando pivot table para somar gasto por unidade
ptable = pd.pivot_table(dataM_unit_gas, index=["id_unidade", "nome_unidade"], values=["valor"], aggfunc=np.sum)

#Salvando essa pivot table num DataFrame
data_gastos_ok = pd.DataFrame(ptable.to_records())

In [4]:
#Carregando arquivo dos docentes (link entre turmas e unidades gestoras)
link_docentes = pd.read_csv("Dados/docentes/docentes.csv",
                           sep=';',
                           usecols=["id_servidor", "id_unidade_lotacao"])

In [58]:
#Carregando arquivos de semestre e linkando

for i in range(2016,2018):
    for j in range(1,3):
        #Carregando strings com o caminho dos dados a serem carregados
        file_matricula = "Dados/matriculas/" + (str(i) + str(j)) + ".csv"
        file_turmas = "Dados/turmas/" + (str(i) + str(j)) + ".csv"
        
        #Carregando arquivo de matriculas de certo semestre para DataFrame
        data_matriculas = pd.read_csv(file_matricula, sep=';',
                                      usecols=["id_turma", "discente", "descricao"])
        #id_turma: necessario para linkar com "turmas"
        #discente: necessario para contagem
        #descricao: o que vamos contar
        
        #Separando valores de descricao irrelevantes para o problema
        tags_indesejadas = ["INDEFERIDO", "AGUARDANDO DEFERIMENTO", "EM ESPERA", 
                            "CUMPRIU", "TRANSFERIDO", "DISPENSADO", "EXCLUIDA"]
        
        #Selecionando apenas os valores nao irrelevantes (relevantes)
        data_matriculas = data_matriculas[~data_matriculas.descricao.isin(tags_indesejadas)]
        
        #Criando mapa para classificar APROVADO ou APROVADO POR NOTA como feedback positivo
        #E classificar todas outras tags como feedback negativo
        map_dict = {}
        for desc in data_matriculas.descricao:
            if desc == "APROVADO POR NOTA" or desc == "APROVADO":
                map_dict[desc] = 1
            else:
                map_dict[desc] = 0
            
        #Atualizar valores das tags com valores mapeados
        data_matriculas.descricao = data_matriculas.descricao.map(map_dict)
        
        #Carregando arquivo de turmas de certo semestre para DataFrame
        data_turmas = pd.read_csv(file_turmas, sep=';',
                                  usecols=["id_turma", "id_docente_interno", "nivel_ensino",
                                           "ano", "periodo","situacao_turma"])
        #id_turma: necessario para linkar com "matriculas"
        #id_docente_interno: necessario para linkar com "docentes"
        #nivel_ensino: restringir para GRADUACAO
        #ano: restringir para ano analisado
        #periodo: restringir para periodo analisado
        #situacao_turma: restringir para turmas consolidadas
        
        #Restringindo a busca para o ano procurado
        data_turmas = data_turmas[data_turmas.ano == i]
        
        #Restringindo a busca para o periodo procurado
        data_turmas = data_turmas[data_turmas.periodo == j]
        
        #Restringindo a busca para o curso superior
        data_turmas = data_turmas[data_turmas.nivel_ensino == "GRADUAÇÃO"]
        
        #Restringindo a busca para turmas consolidadas
        data_turmas = data_turmas[data_turmas.situacao_turma == "CONSOLIDADA"]
        
        #Dando merge nas tabelas turmas e matriculas
        dataM_mat_tur = pd.merge(data_turmas, data_matriculas, on="id_turma")
        
        #Criando pivot table para contar reprovacoes, aprovacoes...
        ptable = pd.pivot_table(dataM_mat_tur, index=["id_turma", "id_docente_interno", "descricao"],
                                values="ano", aggfunc="count").rename(columns={'ano': 'Contagem'})
        
        #Salvando essa pivot table num DataFrame
        data_cont_situ = pd.DataFrame(ptable.to_records())        
        
        #Corrigindo nome de coluna, para possibilitar merge
        data_cont_situ.rename(columns={'id_docente_interno': 'id_servidor'}, inplace=True)
        
        #Executando merge
        dataM_mt_doc = pd.merge(data_cont_situ, link_docentes, on="id_servidor")
        
        #Criando pivot table para agregar contagens por unidade e entao por turma
        ptable = pd.pivot_table(dataM_mt_doc, index=["id_unidade_lotacao", "id_turma", "descricao"],
                               values="Contagem")
        
        #Salvando essa pivot table num DataFrame
        data_cont_unit = pd.DataFrame(ptable.to_records())
        
        #Adicionando nova coluna, referente a porcentagem de valores em "descricao" por "id_turma"
        data_cont_unit["% da turma"] = (data_cont_unit.Contagem / 
                                        data_cont_unit.groupby(["id_unidade_lotacao","id_turma"])
                                        .Contagem.transform(sum) * 100)
        
        ptable = pd.pivot_table(data_cont_unit, index=["id_unidade_lotacao", "id_turma","descricao"],
                        values="% da turma", aggfunc=np.mean)
        
        print(ptable)
        

                                       % da turma
id_unidade_lotacao id_turma descricao            
47                 57573641 0           68.085106
                            1           31.914894
                   57573642 0           10.810811
                            1           89.189189
                   57573643 0            8.163265
                            1           91.836735
                   57573644 0           22.222222
                            1           77.777778
                   57573646 0           57.142857
                            1           42.857143
                   57573647 0           10.000000
                            1           90.000000
                   57573648 0           16.494845
                            1           83.505155
                   57573649 1          100.000000
                   57573651 0           18.181818
                            1           81.818182
                   57573654 0           59.550562


In [98]:
sns.set()
data_gastos
# The four preset contexts, in order of relative size, are paper, notebook, talk, and poster
sns.set_context("notebook")
data_gastos.valor = data_gastos.valor.astype(float)
# plot a univariate distribution of observations.
#sns.kdeplot(int(data_gastos["valor"]))

ValueError: could not convert string to float: 'R$           389,20'